In [17]:
import pandas as pd
import numpy as np
import re
from cleaning import eci18_df_cleaner, eci18_const_corrector

In [18]:
df = pd.read_excel(r"10.Detailed Resulsts.xls")
df.columns = ['num','name','gender','age','candid_categ','party','symbol','gen_votes','post_votes','total_votes','percent_votes']
df.head(5)

,num,name,gender,age,candid_categ,party,symbol,gen_votes,post_votes,total_votes,percent_votes
0,Constituency,1,.,Nippani,TOTAL ELECTORS :,212456,NaN,NaN,NaN,NaN,NaN
1,1,JOLLE SHASHIKALA ANNASAHEB,F,49,GEN,BJP,Lotus,86862.0,144.0,87006.0,50.511173
2,2,KAKASO PANDURANG PATIL,M,63,GEN,INC,Hand,78375.0,125.0,78500.0,45.573030
3,3,KAMAT RAMESH ISHWAR,M,53,SC,BSP,Elephant,2470.0,6.0,2476.0,1.437437
4,4,None of the Above,NaN,NaN,NaN,NOTA,NOTA,1447.0,0.0,1447.0,0.840053


In [19]:
# Getting unique constituencies in a list
constituencies = list(df[df.num=='Constituency']['age'].values)
df = df.astype(str)

In [21]:
# Getting the indices of rows that contain constituency name and the voter turnout info
const_index = list(df[df.num=='Constituency']['num'].index)
turnout_index = list(df[df.num.str.contains('TURNOUT')]['num'].index)

In [22]:
# Extracting the candidates and their gender, age, party, votes, etc. from the main dataframe

temp_dic={
    'constituency':[],
    'name':[],
    'age':[],
    'gender':[],
    'party':[],
    'votes':[],
    'percent_votes':[]
}

for i in range(len(const_index)):
    temp_df = df.iloc[const_index[i]:turnout_index[i]+1].copy(deep=True).reset_index()
    for j in range(temp_df.shape[0]-2):
        temp_dic['constituency'].append(constituencies[i])
        temp_dic['name'].append(temp_df.iloc[1:-1].name.iloc[j].strip())
        temp_dic['age'].append(temp_df.iloc[1:-1].age.iloc[j].strip())
        temp_dic['gender'].append(temp_df.iloc[1:-1].gender.iloc[j].strip())
        temp_dic['party'].append(temp_df.iloc[1:-1].party.iloc[j].strip())
        temp_dic['votes'].append(temp_df.iloc[1:-1].total_votes.iloc[j].strip())
        temp_dic['percent_votes'].append(temp_df.iloc[1:-1].percent_votes.iloc[j].strip())

In [23]:
candidate_df = pd.DataFrame(temp_dic)
candidate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2859 entries, 0 to 2858
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   constituency   2859 non-null   object
 1   name           2859 non-null   object
 2   age            2859 non-null   object
 3   gender         2859 non-null   object
 4   party          2859 non-null   object
 5   votes          2859 non-null   object
 6   percent_votes  2859 non-null   object
dtypes: object(7)
memory usage: 156.5+ KB


In [24]:
# Cleaning and extracting useful info from the dataframe
candidate_df = eci18_df_cleaner(candidate_df)

# Filling missing values (Info about Bye-elections conducted later in 2018)
temp_df = pd.DataFrame({'constituency':['Jayanagar' for _ in range(19)],
                        'name':['Sowmya R','Ajmal Pasha','B N Vijay Kumar','Bojanna N S','D Murugesh','Kalegowda','Lakshminarayana','M.p Nanjesh Kumar','Mohamad Ibrahim','Mohammed Showkathulla Shariff','Moula','Nayeema Khanum','Ravi Krishna Reddy','Shaik Nawaz','Shekar Poojari','Srinath','Syed Zabi','Vijayakumar','Y.k Rahul'],
                        'age':[pd.NA for _ in range(19)],
                        'gender':[pd.NA for _ in range(19)],
                        'party':['INC','IND','BJP','Bhaarateeya Janashakthi Congress','IND','JD(S)','IND','IND','IND','IND','IND','Prajaa Raita Raajya Paksha','IND','Samanya Janatha Party (Loktantrik)','Karnataka Pragnyavantha Janatha Party','Kannada Paksha','All India Mahila Empowerment Party','IND','Rashtriya Manav Vikas Party'],
                        'votes':[54458,35,51571,88,132,817,218,98,62,121,189,246,1861,35,101,38,60,568,44],
                        'percent_votes':[pd.NA for _ in range(19)],
                       'eci_constituency':['Jayanagar' for _ in range(19)],
                       'eci_constituency_category':['GEN' for _ in range(19)]})
candidate_df = pd.concat([candidate_df,temp_df],ignore_index=True,axis=0)
candidate_df = candidate_df.drop(['constituency'],axis='columns').rename(columns={'eci_constituency':'constituency','eci_constituency_category':'constituency_category'})
candidate_df

,name,age,gender,party,votes,percent_votes,constituency,constituency_category
0,Jolle Shashikala Annasaheb,49,F,BJP,87006,50.511173,Nippani,GEN
1,Kakaso Pandurang Patil,63,M,INC,78500,45.57303,Nippani,GEN
2,Kamat Ramesh Ishwar,53,M,BSP,2476,1.437437,Nippani,GEN
3,NOTA,<NA>,NaN,NOTA,1447,0.840053,Nippani,GEN
4,Rohini Shrimant Dixit,35,F,AIMEP,1353,0.785482,Nippani,GEN
...,...,...,...,...,...,...,...,...
2873,Shekar Poojari,<NA>,<NA>,Karnataka Pragnyavantha Janatha Party,101,<NA>,Jayanagar,GEN
2874,Srinath,<NA>,<NA>,Kannada Paksha,38,<NA>,Jayanagar,GEN
2875,Syed Zabi,<NA>,<NA>,All India Mahila Empowerment Party,60,<NA>,Jayanagar,GEN
2876,Vijayakumar,<NA>,<NA>,IND,568,<NA>,Jayanagar,GEN


In [25]:
candidate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2878 entries, 0 to 2877
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   name                   2878 non-null   object
 1   age                    2636 non-null   object
 2   gender                 2636 non-null   object
 3   party                  2878 non-null   object
 4   votes                  2878 non-null   Int64 
 5   percent_votes          2859 non-null   object
 6   constituency           2878 non-null   object
 7   constituency_category  2878 non-null   object
dtypes: Int64(1), object(7)
memory usage: 182.8+ KB


In [26]:
# Converting the candidates dataframe to csv file.
candidate_df.to_csv('candidates2018ECI.csv')

In [27]:
# Filtering the data about the constituency (name, category, eligible voters, voter turnout) from the main dataframe.

const_df = df.loc[sorted(const_index),['age','party']]
const_df.age = const_df.age.str.removesuffix('  (SC)').str.removesuffix('  (ST)').str.strip().str.replace(u'\xa0',' ')
const_df.party = const_df.party.astype('Int32')
const_df = const_df.reset_index().rename(columns={'age':'constituency','party':'total_electors'})[['constituency','total_electors']]
const_df.constituency = const_df.constituency.apply(eci18_const_corrector)

turn_df = df.loc[sorted(turnout_index),['candid_categ','party']]
turn_df = turn_df.rename(columns={'candid_categ':'voter_turnout','party':'voter_turnout_percent'}).reset_index()[['voter_turnout','voter_turnout_percent']]
turn_df.voter_turnout = turn_df.voter_turnout.astype('Int32')
turn_df.voter_turnout_percent = turn_df.voter_turnout_percent.astype('Float64')

const_df = pd.concat([const_df,turn_df],axis=1)
const_df

,constituency,total_electors,voter_turnout,voter_turnout_percent
0,Nippani,212456,172251,81.076082
1,Chikkodi-Sadalga,211083,179201,84.895989
2,Athani,214688,170565,79.44785
3,Kagwad,182173,145735,79.998134
4,Kudachi,180509,136653,75.704259
...,...,...,...,...
218,T.Narasipur,198441,155286,78.252982
219,Hanur,207662,170333,82.024155
220,Kollegal,211555,168871,79.823687
221,Chamarajanagar,206162,167111,81.0581


In [28]:
const_df

,constituency,total_electors,voter_turnout,voter_turnout_percent
0,Nippani,212456,172251,81.076082
1,Chikkodi-Sadalga,211083,179201,84.895989
2,Athani,214688,170565,79.44785
3,Kagwad,182173,145735,79.998134
4,Kudachi,180509,136653,75.704259
...,...,...,...,...
218,T.Narasipur,198441,155286,78.252982
219,Hanur,207662,170333,82.024155
220,Kollegal,211555,168871,79.823687
221,Chamarajanagar,206162,167111,81.0581


In [29]:
# Filtering the winners from the candidates dataframe and merging them with the constituency dataframe
temp_df = candidate_df.groupby('constituency').apply(lambda x: x.sort_values('votes',ascending=False).head(1).squeeze().drop('constituency'))
const_df = temp_df.merge(const_df,on='constituency')
const_df = const_df.drop(['voter_turnout_percent'],axis=1)

# Filling the missing values
temp_df = pd.DataFrame({'name':['Sowmya R'],'age':[pd.NA],'gender':[pd.NA],
                        'party':['INC'],'votes':[54458],'percent_votes':[54458],
                       'constituency':['Jayanagar'],'constituency_category':['GEN'],
                       'total_electors':[203184],'voter_turnout':[111584]})
const_df = pd.concat([temp_df,const_df],ignore_index=True)

const_df.head(5)

,name,age,gender,party,votes,percent_votes,constituency,constituency_category,total_electors,voter_turnout
0,Sowmya R,<NA>,<NA>,INC,54458,54458,Jayanagar,GEN,203184,111584
1,M Y Patil,77,M,INC,71735,47.916611,Afzalpur,GEN,220007,149708
2,Guttedar Subhash Rukmayya,66,M,BJP,76815,47.827332,Aland,GEN,233841,160609
3,B.Shivanna,50,M,INC,113894,50.024377,Anekal,SC,359550,227677
4,Balachandra Lakshmanarao Jarakiholi,51,M,BJP,96144,54.338292,Arabhavi,GEN,231864,176936


In [30]:
const_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   name                   224 non-null    object
 1   age                    223 non-null    object
 2   gender                 223 non-null    object
 3   party                  224 non-null    object
 4   votes                  224 non-null    int64 
 5   percent_votes          224 non-null    object
 6   constituency           224 non-null    object
 7   constituency_category  224 non-null    object
 8   total_electors         224 non-null    Int64 
 9   voter_turnout          224 non-null    Int64 
dtypes: Int64(2), int64(1), object(7)
memory usage: 18.1+ KB


In [31]:
# Converting the constituency dataframe to a csv file.
const_df.to_csv('constituency2018ECI.csv')